In [24]:
import pandas as pd
from datetime import datetime
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

# Завантаження даних
train_data = pd.read_csv('D:/GoIT_ML/DZ_MDS_3/dz_mod_8/mod_04_hw_train_data.csv')
valid_data = pd.read_csv('D:/GoIT_ML/DZ_MDS_3/dz_mod_8/mod_04_hw_valid_data.csv')

# Видалення колонок Name, Phone_Number
train_data_cleaned = train_data.drop(columns=['Name', 'Phone_Number'])
valid_data_cleaned = valid_data.drop(columns=['Name', 'Phone_Number'])

train_data_cleaned = train_data_cleaned.dropna()

# Функція для обчислення віку
def calculate_age(birthdate):
    birthdate = datetime.strptime(birthdate, '%d/%m/%Y')
    today = datetime.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

# Додавання віку і видалення дати народження
train_data_cleaned['Age'] = train_data_cleaned['Date_Of_Birth'].apply(calculate_age)
valid_data_cleaned['Age'] = valid_data_cleaned['Date_Of_Birth'].apply(calculate_age)
train_data_cleaned = train_data_cleaned.drop(columns=['Date_Of_Birth', 'Age'])
valid_data_cleaned = valid_data_cleaned.drop(columns=['Date_Of_Birth', 'Age'])

# Перевірка наявності пропущених значень
print("Перевірка пропущених значень у тренувальних даних:")
print(train_data_cleaned.isnull().sum())

print("Перевірка пропущених значень у валідаційних даних:")
print(valid_data_cleaned.isnull().sum())

# # Обробка пропущених значень лише для тренувальних даних
# for col in ['Experience', 'Qualification', 'Role', 'Cert']:
#     if train_data_cleaned[col].isnull().sum() > 0:
#         if train_data_cleaned[col].dtype == 'object':
#             train_data_cleaned[col] = train_data_cleaned[col].fillna(train_data_cleaned[col].mode()[0])
#         else:
#             train_data_cleaned[col] = train_data_cleaned[col].fillna(train_data_cleaned[col].mean())

# 1. Дискретизація числових ознак

# kbins = KBinsDiscretizer(encode='ordinal', strategy='uniform')
# train_data_cleaned[numeric_features] = kbins.fit_transform(train_data_cleaned[numeric_features])
# valid_data_cleaned[numeric_features] = kbins.transform(valid_data_cleaned[numeric_features])

# Візуалізація розподілу кожної числової ознаки до нормалізації
# for feature in numeric_features:
#     plt.figure(figsize=(10, 6))
#     sns.histplot(train_data_cleaned[feature], kde=True, bins=20)
#     plt.title(f'Розподіл ознаки {feature} до нормалізації')
#     plt.xlabel(f'{feature} (Raw)')
#     plt.ylabel('Frequency')
#     plt.show()


Перевірка пропущених значень у тренувальних даних:
Experience       0
Qualification    0
University       0
Role             0
Cert             0
Salary           0
dtype: int64
Перевірка пропущених значень у валідаційних даних:
Experience       0
Qualification    0
University       0
Role             0
Cert             0
Salary           0
dtype: int64


In [25]:
numeric_features = ['Experience']
category_cols = ['Qualification', 'University', 'Role', 'Cert']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    #('power', PowerTransformer()),  # Нормалізація до нормального розподілу
    #('scaler', StandardScaler())    # Масштабування
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target_enc', TargetEncoder()),  # Нормалізація до нормального розподілу
])

# Об'єднання трансформацій у ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, category_cols)
    ]).set_output(transform='pandas')


# 2. Нормалізація числових ознак за допомогою PowerTransformer та StandardScaler
#all_numeric_features = ['Experience'] + [col for col in train_data_cleaned.select_dtypes(include=['number']).columns if col not in ['Salary']]


power_transf = PowerTransformer()

# Масштабування і кодування тренувальних та валідаційних наборів даних
X_train = train_data_cleaned.drop(columns=['Salary'])
X_valid = valid_data_cleaned.drop(columns=['Salary'])

# Цільова змінна (заробітна плата)
y_train = train_data_cleaned['Salary']
y_valid = valid_data_cleaned['Salary']

X_train_transformed = preprocessor.fit_transform(X_train, y_train)
X_valid_transformed = preprocessor.transform(X_valid)

X_train_transformed = power_transf.fit_transform(X_train_transformed)
X_valid_transformed = power_transf.transform(X_valid_transformed)




In [26]:

# # Додаткова перевірка на наявність некоректних значень після перетворення
# print("Перевірка наявності текстових даних після кодування:")
# for col in train_data_cleaned.columns:
#     if train_data_cleaned[col].dtype == 'object':
#         print(f"Текстові значення у стовпці '{col}': {train_data_cleaned[col].unique()}")

# # Дослідницький аналіз даних (EDA)
# # Розподіл заробітної плати
# plt.figure(figsize=(10, 6))
# sns.histplot(train_data_cleaned['Salary'], kde=True)
# plt.title('Розподіл заробітної плати (Salary)')
# plt.xlabel('Заробітна плата')
# plt.ylabel('Кількість')
# plt.show()

# # Кореляційна матриця для числових ознак
# plt.figure(figsize=(10, 8))
# correlation_matrix = train_data_cleaned.corr()
# sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
# plt.title('Кореляційна матриця тренувальних даних')
# plt.show()


# # Залежність заробітної плати від досвіду
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x=train_data_cleaned['Experience'], y=train_data_cleaned['Salary'])
# plt.title('Залежність заробітної плати від досвіду')
# plt.xlabel('Досвід (років)')
# plt.ylabel('Заробітна плата')
# plt.show()

# # Розподіл заробітної плати по ролям
# plt.figure(figsize=(10, 6))
# sns.boxplot(x='Role', y='Salary', data=train_data_cleaned)
# plt.title('Розподіл заробітної плати по ролям')
# plt.xlabel('Роль')
# plt.ylabel('Заробітна плата')
# plt.show()

In [27]:
# Пошук оптимального значення n_neighbors
best_mape = float('inf')  # Найменший MAPE для вибору оптимального значення
best_neighbors = 5        # Початкове значення n_neighbors
results = []

# Перевірка n_neighbors від 1 до 20
for n in range(1, 21):
    # Побудова моделі KNeighborsRegressor із зазначеним n_neighbors
    knn_model = KNeighborsRegressor(n_neighbors=n)

    # Навчання моделі
    knn_model.fit(X_train_transformed, y_train)

    # Прогнозування на валідаційному наборі
    y_valid_pred_knn = knn_model.predict(X_valid_transformed)

    # Оцінка моделі
    mae_knn = mean_absolute_error(y_valid, y_valid_pred_knn)
    mape_knn = mean_absolute_percentage_error(y_valid, y_valid_pred_knn)
    r2_knn = r2_score(y_valid, y_valid_pred_knn)

    # Збереження результатів для кожного n_neighbors
    results.append((n, mae_knn, mape_knn, r2_knn))

    # Оновлення найкращого n_neighbors
    if mape_knn < best_mape:
        best_mape = mape_knn
        best_neighbors = n

    print(f"n_neighbors = {n}: MAE = {mae_knn:.2f}, MAPE = {mape_knn:.2%}, R2 = {r2_knn:.2f}")

# Виведення найкращого значення n_neighbors
print(f"\nНайкращий результат: n_neighbors = {best_neighbors} із MAPE = {best_mape:.2%}")

# Побудова моделі з найкращим значенням n_neighbors
knn_model = KNeighborsRegressor(n_neighbors=best_neighbors)
knn_model.fit(X_train_transformed, y_train)

# Прогнозування з найкращою моделлю
y_valid_pred_knn = knn_model.predict(X_valid_transformed)

# Фінальна оцінка моделі
mae_knn = mean_absolute_error(y_valid, y_valid_pred_knn)
mape_knn = mean_absolute_percentage_error(y_valid, y_valid_pred_knn)
r2_knn = r2_score(y_valid, y_valid_pred_knn)

print(f"Фінальний результат з оптимальним n_neighbors = {best_neighbors}:")
print(f"Mean Absolute Error (MAE): {mae_knn:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_knn:.2%}")
print(f"R-squared (R2): {r2_knn:.2f}")

n_neighbors = 1: MAE = 4128.57, MAPE = 4.99%, R2 = 0.87
n_neighbors = 2: MAE = 6157.14, MAPE = 7.05%, R2 = 0.80
n_neighbors = 3: MAE = 3804.76, MAPE = 4.04%, R2 = 0.85
n_neighbors = 4: MAE = 4600.00, MAPE = 5.00%, R2 = 0.77
n_neighbors = 5: MAE = 4471.43, MAPE = 4.96%, R2 = 0.82
n_neighbors = 6: MAE = 3395.24, MAPE = 3.63%, R2 = 0.88
n_neighbors = 7: MAE = 2834.69, MAPE = 3.13%, R2 = 0.90
n_neighbors = 8: MAE = 2666.07, MAPE = 3.08%, R2 = 0.95
n_neighbors = 9: MAE = 3538.10, MAPE = 4.09%, R2 = 0.93
n_neighbors = 10: MAE = 3492.86, MAPE = 4.10%, R2 = 0.93
n_neighbors = 11: MAE = 3746.75, MAPE = 4.43%, R2 = 0.92
n_neighbors = 12: MAE = 3845.24, MAPE = 4.63%, R2 = 0.92
n_neighbors = 13: MAE = 3445.05, MAPE = 4.14%, R2 = 0.93
n_neighbors = 14: MAE = 3913.27, MAPE = 4.67%, R2 = 0.90
n_neighbors = 15: MAE = 3633.33, MAPE = 4.29%, R2 = 0.91
n_neighbors = 16: MAE = 4250.89, MAPE = 4.96%, R2 = 0.89
n_neighbors = 17: MAE = 4980.67, MAPE = 5.75%, R2 = 0.87
n_neighbors = 18: MAE = 4859.52, MAPE = 